In this notebook we want to test automatising the computation of matrix elements as functions of position, and F^H (so to be able to use Krylov iterative solvers), permanent storage in an according file containing all symbolic results (this will only be generated once per Green's function), and the subsequent usage of this file by the linear solver.

On the note of a general Green's function, have to write the matrix elements more general. In previous computation I have used a lot of the symmetries of the free space Green's function.